In [1]:
import torch

torch.__version__

'2.2.0'

In [6]:
import sqlite3

con = sqlite3.connect('../gold.db')
cur = con.cursor()

str(cur.execute("pragma table_info('stock');").fetchall())

"[(0, 'idx', 'INT', 0, None, 0), (1, 'Date_', 'date', 0, None, 0), (2, 'Close', 'float', 0, None, 0), (3, 'Volume', 'float', 0, None, 0), (4, 'Open', 'float', 0, None, 0), (5, 'High', 'float', 0, None, 0), (6, 'Low', 'float', 0, None, 0)]"

In [2]:
cur.execute('select Date_ from stock order by Date_ desc limit 2;').fetchall()

[('Date',), ('2024-01-19',)]

In [7]:
query = '''
SELECT Close
FROM stock
WHERE Date_ >= DATE('now', '-3 month')
ORDER BY Date_ DESC;
'''

import pandas as pd
df = pd.read_sql(query, con)

In [11]:
str(df.columns)

"Index(['Close'], dtype='object')"

In [2]:
import subprocess
import time
from IPython.display import clear_output

def get_gpu_usage(interval=1, duration=10):
    """
    Monitors the GPU usage by repeatedly calling nvidia-smi.
    
    :param interval: Interval in seconds between each update.
    :param duration: Total duration in seconds for which to monitor.
    """
    end_time = time.time() + duration
    while time.time() < end_time:
        # Run nvidia-smi command
        result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE)
        
        # Clear the output to avoid clutter
        clear_output(wait=True)
        
        # Print the result
        print(result.stdout.decode("utf-8"))
        
        # Wait for the specified interval
        time.sleep(interval)

# Example usage
get_gpu_usage(interval=0.5, duration=100)

Mon Feb 12 13:26:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:5E:00.0 Off |                  Off |
| 30%   51C    P0              79W / 300W |      2MiB / 49140MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

KeyboardInterrupt: 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-70B-Chat-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-64g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# print("\n\n*** Generate:")

# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    # temperature=0.3,
    # top_p=0.85,
    top_k=40,
    repetition_penalty=1.1
)

# print(pipe(prompt_template)[0]['generated_text'])

/home/ppathi/miniconda3/envs/bot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.


*** Pipeline:


In [4]:
prompt = "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1."
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature. 
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
[/INST]
<<USER>>
{prompt}
<</USER>>
'''

print(pipe(prompt_template)[0]['generated_text'])

[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
Let's work this out in a step by step way to be sure we have the right answer. 
<</SYS>>
[/INST]
<<USER>>
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
<</USER>>

I apologize for any confusion, but the statement you provided doesn't make mathematical sense. The sum of those six odd numbers is 30, which is an odd number, not an even number. Can you please rephrase or provide more context so I can better understand what you're asking?
